In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn as sk
import skimage.feature
import imutils

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error
from random import shuffle
from PIL import Image

plt.style.use("ggplot")

In [4]:
data = []
Positives = os.listdir("Concrete_Crack_Images_for_Classification/Positive")
Negatives = os.listdir("Concrete_Crack_Images_for_Classification/Negative")

In [5]:
for img in Positives: # This will take a while
    try:
        # Read the images
        print("Adding Positive Image ", img)
        bgr_image = cv2.imread("Concrete_Crack_Images_for_Classification/Positive/" + str(img)) 
        gry_img = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY) # Convert image to greyscale
        edged = imutils.auto_canny(gry_img)
        cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        if cnts:
            c = max(cnts, key=cv2.contourArea)
            # extract the logo of the car and resize it to a canonical width
            # and height
            (x, y, w, h) = cv2.boundingRect(c)
            logo = gry_img[y:y + h, x:x + w]
            logo = cv2.resize(logo, (200, 100))
 
            # extract Histogram of Oriented Gradients from the logo
            H = skimage.feature.hog(logo, orientations=9, pixels_per_cell=(10, 10), cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1")
            data.append([H, 1]) # Label 1 for positive
    except AttributeError:
        print("")

for img in Negatives: # This will take a while
    try:
        print("Adding Negative Image ", img)
        # Read the images
        bgr_image = cv2.imread("Concrete_Crack_Images_for_Classification/Negative/" + str(img)) 
        gry_img = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY) # Convert image to greyscale
        edged = imutils.auto_canny(gry_img)
        cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        if cnts:
            c = max(cnts, key=cv2.contourArea)
            # extract the logo of the car and resize it to a canonical width
            # and height
            (x, y, w, h) = cv2.boundingRect(c)
            logo = gry_img[y:y + h, x:x + w]
            logo = cv2.resize(logo, (200, 100))
 
            # extract Histogram of Oriented Gradients from the logo
            H = skimage.feature.hog(logo, orientations=9, pixels_per_cell=(10, 10), cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1")
            data.append([H, 0]) # Label 0 for negative
    except AttributeError:
        print("")

g
Adding Negative Image  19395.jpg
Adding Negative Image  19396.jpg
Adding Negative Image  19397.jpg
Adding Negative Image  19398.jpg
Adding Negative Image  19399.jpg
Adding Negative Image  19400.jpg
Adding Negative Image  19401.jpg
Adding Negative Image  19402.jpg
Adding Negative Image  19403.jpg
Adding Negative Image  19404.jpg
Adding Negative Image  19405.jpg
Adding Negative Image  19406.jpg
Adding Negative Image  19407.jpg
Adding Negative Image  19408.jpg
Adding Negative Image  19409.jpg
Adding Negative Image  19410.jpg
Adding Negative Image  19411.jpg
Adding Negative Image  19412.jpg
Adding Negative Image  19413.jpg
Adding Negative Image  19414.jpg
Adding Negative Image  19415.jpg
Adding Negative Image  19416.jpg
Adding Negative Image  19417.jpg
Adding Negative Image  19418.jpg
Adding Negative Image  19419.jpg
Adding Negative Image  19420.jpg
Adding Negative Image  19421.jpg
Adding Negative Image  19422.jpg
Adding Negative Image  19423.jpg
Adding Negative Image  19424.jpg
Adding N

In [6]:
shuffle(data) # Shuffle the data 

feature_vector_X = [data[vec][0].flatten() for vec in range(len(data))]
data_labels_y = [data[label][1] for label in range(len(data))]

X_train, X_test, y_train, y_test = train_test_split(feature_vector_X, data_labels_y, random_state=0, test_size=0.2)

In [17]:
estimator = sk.linear_model.RidgeClassifier(alpha=1)
model = estimator.fit(X_train, y_train) # This may take a while

In [18]:
predictions = model.predict(X_test)
print(sk.metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.86      0.85      0.85      2676
           1       0.90      0.90      0.90      3985

    accuracy                           0.88      6661
   macro avg       0.88      0.88      0.88      6661
weighted avg       0.88      0.88      0.88      6661

